In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Cargar los datos de indicadores_economicos.csv
df_indicadores = pd.read_csv('indicadores_economicos.csv')

# Convertir la columna 'Year' a tipo fecha y extraer solo el año como entero
df_indicadores['Year'] = pd.to_datetime(df_indicadores['Year']).dt.year

# Columnas relevantes seleccionadas manualmente
columnas_relevantes = ['GDP (current US$) [NY.GDP.MKTP.CD]', 'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]', 'Gross capital formation (% of GDP) [NE.GDI.TOTL.ZS]', 
                       'Inflation, GDP deflator (annual %) [NY.GDP.DEFL.KD.ZG]', 'Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]',
                       'Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]', 'Total natural resources rents (% of GDP) [NY.GDP.TOTL.RT.ZS]']

# Agrupar por país
grupos_pais = df_indicadores.groupby('Country Name')

# Definir la función de ajuste para cada columna relevante
def ajustar_columna(datos):
    for columna in columnas_relevantes:
        # Obtener los valores conocidos y sus respectivos años
        x_known = datos.loc[~datos[columna].isna(), 'Year'].values.reshape(-1, 1)
        y_known = datos.loc[~datos[columna].isna(), columna].values
        
        # Si no hay valores conocidos, no se puede ajustar el modelo, así que continuamos con la siguiente columna
        if len(x_known) == 0:
            continue
        
        # Crear el modelo de regresión lineal
        modelo_lineal = LinearRegression()
        
        # Ajustar el modelo a los datos conocidos
        modelo_lineal.fit(x_known, y_known)
               
        # Obtener los años con valores perdidos en la columna actual
        x_pred = datos[datos[columna].isna()]['Year'].values.reshape(-1, 1)
        
        if len(x_pred) == 0:
            continue
                
        # Predecir los valores perdidos
        y_pred = modelo_lineal.predict(x_pred)
        
        # Actualizar los valores perdidos en el DataFrame original con los valores ajustados
        datos.loc[datos[columna].isna(), columna] = y_pred.flatten()
    
    return datos

# Aplicar la función de ajuste a cada grupo (país) y columna relevante
df_indicadores = grupos_pais.apply(ajustar_columna)

# Guardar el DataFrame con los valores ajustados en un nuevo archivo CSV
df_indicadores.to_csv('indicadores_economicos_ajustados.csv', index=False)
